<a href="https://colab.research.google.com/github/ashiqurrahmankhan21st/NODE/blob/main/NODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# STEP 1: Install Required Packages
!pip install torch scikit-learn pandas
!pip install optuna
# STEP 3: Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from torch.utils.data import DataLoader, TensorDataset



import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [42]:
df = pd.read_csv("https://raw.githubusercontent.com/ashiqurrahmankhan21st/NODE/refs/heads/main/FFF%20Data.csv")
df.head()

,Extruder_temperature,Printing_speed,Infill_density,Layer_height,Wall_layer,Infill_pattern,Average_tensile_strength
0,215,90,20,0.2,3,Diamond,18.775
1,215,90,10,0.2,3,Diamond,7.625
2,215,90,5,0.2,3,Diamond,4.300
3,215,90,20,0.2,3,Diamond,18.775
4,215,90,10,0.2,3,Diamond,7.625


In [43]:
df.shape

(92, 7)

In [44]:
# Load the dataset
df = pd.read_csv("https://raw.githubusercontent.com/ashiqurrahmankhan21st/NODE/refs/heads/main/FFF%20Data.csv")
X = df.drop(columns=["Average_tensile_strength"])
y = df["Average_tensile_strength"]

In [45]:
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

In [46]:
# Define categorical and numerical columns
categorical_cols = ["Infill_pattern"]
numeric_cols = [col for col in X.columns if col not in categorical_cols]

In [47]:
# Preprocessing
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(sparse_output=False), categorical_cols)
])

X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [48]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=16, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=16)

In [49]:
# Oblivious Decision Tree Layer
class ObliviousDecisionTreeLayer(nn.Module):
    def __init__(self, input_dim, num_trees, tree_depth):
        super().__init__()
        self.num_leafs = 2 ** tree_depth
        self.feature_selectors = nn.Parameter(torch.randn(num_trees, tree_depth, input_dim))
        self.feature_bias = nn.Parameter(torch.zeros(num_trees, tree_depth))
        self.leaf_scores = nn.Parameter(torch.randn(num_trees, self.num_leafs))

    def forward(self, x):
        # Reshape input for batch matrix multiplication
        x_expanded = x.unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, input_dim]
        w = self.feature_selectors.unsqueeze(0)   # [1, num_trees, tree_depth, input_dim]
        logits = (x_expanded * w).sum(dim=-1) + self.feature_bias.unsqueeze(0)  # [batch, num_trees, tree_depth]
        gates = torch.sigmoid(logits)
        for d in range(gates.shape[2]):
            if d == 0:
                probs = torch.stack([1 - gates[:, :, d], gates[:, :, d]], dim=-1)
            else:
                probs = torch.cat([
                    probs * (1 - gates[:, :, d]).unsqueeze(-1),
                    probs * gates[:, :, d].unsqueeze(-1)
                ], dim=-1)
        return torch.sum(probs * self.leaf_scores.unsqueeze(0), dim=-1)

In [55]:
# Full NODE model
class NODE(nn.Module):
    def __init__(self, input_dim, num_trees=8, depth=3):
        super().__init__()
        self.tree1 = ObliviousDecisionTreeLayer(input_dim=input_dim, num_trees=num_trees, tree_depth=depth)
        self.tree2 = ObliviousDecisionTreeLayer(input_dim=num_trees, num_trees=num_trees, tree_depth=depth)
        self.tree3 = ObliviousDecisionTreeLayer(input_dim=num_trees, num_trees=num_trees, tree_depth=depth)
        self.out = nn.Linear(num_trees, 1)

    def forward(self, x):
        x = self.tree1(x)
        x = self.tree2(x)
        x = self.tree3(x)
        return self.out(x)

In [56]:
# Initialize model
input_dim = X_train.shape[1]
model = NODE(input_dim=input_dim, num_trees=32, depth=4) #'num_trees': 32, 'depth': 4, 'lr': 0.00994460910700778
optimizer = torch.optim.Adam(model.parameters(), lr=0.00994460910700778)
criterion = nn.MSELoss()

In [57]:
# Training loop
for epoch in range(100):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}")

Epoch 1: Train Loss = 4212.0243
Epoch 2: Train Loss = 3847.5358
Epoch 3: Train Loss = 3460.1953
Epoch 4: Train Loss = 2978.8969
Epoch 5: Train Loss = 2389.2361
Epoch 6: Train Loss = 1742.7714
Epoch 7: Train Loss = 1132.8775
Epoch 8: Train Loss = 659.0740
Epoch 9: Train Loss = 361.4209
Epoch 10: Train Loss = 215.6336
Epoch 11: Train Loss = 152.4605
Epoch 12: Train Loss = 116.2104
Epoch 13: Train Loss = 96.3927
Epoch 14: Train Loss = 71.1989
Epoch 15: Train Loss = 56.2371
Epoch 16: Train Loss = 51.2663
Epoch 17: Train Loss = 45.2870
Epoch 18: Train Loss = 40.5118
Epoch 19: Train Loss = 37.2660
Epoch 20: Train Loss = 35.1721
Epoch 21: Train Loss = 33.3349
Epoch 22: Train Loss = 32.0172
Epoch 23: Train Loss = 30.8635
Epoch 24: Train Loss = 29.9279
Epoch 25: Train Loss = 29.2121
Epoch 26: Train Loss = 28.5154
Epoch 27: Train Loss = 27.9054
Epoch 28: Train Loss = 27.4077
Epoch 29: Train Loss = 26.9632
Epoch 30: Train Loss = 26.5673
Epoch 31: Train Loss = 26.2653
Epoch 32: Train Loss = 26.010

In [58]:
# Evaluation
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)
    test_loss = criterion(preds, y_test_tensor).item()
    print(f"\nTest MSE: {test_loss:.4f}")



Test MSE: 49.6380


In [54]:
import optuna
from sklearn.metrics import mean_squared_error

# Objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    num_trees = trial.suggest_int("num_trees", 4, 32)
    depth = trial.suggest_int("depth", 2, 5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    # Define model with suggested hyperparameters
    model = NODE(input_dim=X_train.shape[1], num_trees=num_trees, depth=depth)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # Train for a few epochs
    for epoch in range(10):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.numpy(), preds.numpy())
    return mse

# Run Bayesian optimization using Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Best result
best_params = study.best_params
best_score = study.best_value

(best_params, best_score)


[I 2025-06-08 00:11:23,046] A new study created in memory with name: no-name-5fcdad50-5f27-4626-bf8a-b20f2fded9cd
[I 2025-06-08 00:11:23,373] Trial 0 finished with value: 189.34512329101562 and parameters: {'num_trees': 30, 'depth': 3, 'lr': 0.00822469325296412}. Best is trial 0 with value: 189.34512329101562.
[I 2025-06-08 00:11:23,692] Trial 1 finished with value: 761.4637451171875 and parameters: {'num_trees': 27, 'depth': 5, 'lr': 0.0028291179736444705}. Best is trial 0 with value: 189.34512329101562.
[I 2025-06-08 00:11:24,098] Trial 2 finished with value: 463.753662109375 and parameters: {'num_trees': 30, 'depth': 4, 'lr': 0.0049953999517347095}. Best is trial 0 with value: 189.34512329101562.
[I 2025-06-08 00:11:24,309] Trial 3 finished with value: 904.064453125 and parameters: {'num_trees': 28, 'depth': 3, 'lr': 0.00021246416371760375}. Best is trial 0 with value: 189.34512329101562.
[I 2025-06-08 00:11:24,472] Trial 4 finished with value: 634.716552734375 and parameters: {'num

({'num_trees': 22, 'depth': 3, 'lr': 0.0099569420623071}, 132.58473205566406)

In [65]:
best_params['depth']

22

In [67]:
# Initialize model
input_dim = X_train.shape[1]
model = NODE(input_dim=input_dim, num_trees=best_params['num_trees'], depth=best_params['depth']) #'num_trees': 32, 'depth': 4, 'lr': 0.00994460910700778
optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'])
criterion = nn.MSELoss()

# Training loop
for epoch in range(100):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)
    test_loss = criterion(preds, y_test_tensor).item()
    print(f"\nTest MSE: {test_loss:.4f}")

Epoch 1: Train Loss = 4109.4975
Epoch 2: Train Loss = 3739.5243
Epoch 3: Train Loss = 3368.6086
Epoch 4: Train Loss = 2966.4324
Epoch 5: Train Loss = 2527.5664
Epoch 6: Train Loss = 2071.6934
Epoch 7: Train Loss = 1623.1640
Epoch 8: Train Loss = 1227.6982
Epoch 9: Train Loss = 911.3263
Epoch 10: Train Loss = 661.3268
Epoch 11: Train Loss = 469.6521
Epoch 12: Train Loss = 320.1546
Epoch 13: Train Loss = 213.7352
Epoch 14: Train Loss = 148.9358
Epoch 15: Train Loss = 119.1144
Epoch 16: Train Loss = 100.5185
Epoch 17: Train Loss = 93.2676
Epoch 18: Train Loss = 90.4434
Epoch 19: Train Loss = 85.9043
Epoch 20: Train Loss = 79.1281
Epoch 21: Train Loss = 72.7636
Epoch 22: Train Loss = 67.4568
Epoch 23: Train Loss = 60.7032
Epoch 24: Train Loss = 54.1709
Epoch 25: Train Loss = 49.0842
Epoch 26: Train Loss = 45.0192
Epoch 27: Train Loss = 42.0317
Epoch 28: Train Loss = 39.8705
Epoch 29: Train Loss = 38.2870
Epoch 30: Train Loss = 36.9415
Epoch 31: Train Loss = 35.7199
Epoch 32: Train Loss = 3

In [68]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test_tensor.numpy(), preds.numpy())
print("R^2 Score:", r2)


R^2 Score: 0.25046950578689575
